Test des webservices communautaires

In [112]:
from kiblib.utils.conf import Config
from requests.auth import HTTPBasicAuth

from kiblib.utils.date import get_date_and_time

import pandas as pd
import requests
import json
import datetime

In [69]:
# gestion de l'authentification
ws_conf = Config().get_config_webservice()
#ws_conf['base'] = 'https://webservice-pp-mediatheque.ville-roubaix.fr'
basic_auth = HTTPBasicAuth(ws_conf['user'], ws_conf['pwd'])

# Obtenir les informations d'un adhérent

In [70]:
patron_id = 3745
url = f"{ws_conf['base']}/api/v1/patrons/{patron_id}"
response = requests.get(url, auth=basic_auth)
response.json()

{'address': '17 RUE FERDINAND BUISSON',
 'address2': '',
 'altaddress_address': '',
 'altaddress_address2': '',
 'altaddress_city': '',
 'altaddress_country': '',
 'altaddress_email': '',
 'altaddress_notes': '',
 'altaddress_phone': '',
 'altaddress_postal_code': '',
 'altaddress_state': '',
 'altaddress_street_number': '',
 'altaddress_street_type': '',
 'altcontact_address': '',
 'altcontact_address2': '',
 'altcontact_city': '',
 'altcontact_country': '',
 'altcontact_firstname': '',
 'altcontact_phone': '',
 'altcontact_postal_code': '',
 'altcontact_state': '',
 'altcontact_surname': '',
 'anonymized': False,
 'autorenew_checkouts': True,
 'cardnumber': 'X0002181004',
 'category_id': 'MEDP',
 'check_previous_checkout': 'inherit',
 'city': 'HAZEBROUCK',
 'country': '',
 'date_enrolled': '2011-09-02',
 'date_of_birth': '1981-01-28',
 'date_renewed': '2024-08-03',
 'email': 'fpichenot@ville-roubaix.fr',
 'expiry_date': '2025-08-03',
 'fax': '',
 'firstname': 'François',
 'gender': '

# Obtenir les données de tous les adhérents

In [83]:
dfs = []

url = f"{ws_conf['base']}/api/v1/patrons?_per_page=500"
response = requests.get(url, auth=basic_auth)
dfs.append(pd.DataFrame(response.json()))

In [84]:
while 'next' in response.links:
    print(response.links['next']['url'])
    response = requests.get(response.links['next']['url'], auth=basic_auth)
    dfs.append(pd.DataFrame(response.json()))

http://cataloguekoha.ntrbx.local/api/v1/patrons?_per_page=500&_page=2
http://cataloguekoha.ntrbx.local/api/v1/patrons?_per_page=500&_page=3
http://cataloguekoha.ntrbx.local/api/v1/patrons?_page=4&_per_page=500
http://cataloguekoha.ntrbx.local/api/v1/patrons?_per_page=500&_page=5
http://cataloguekoha.ntrbx.local/api/v1/patrons?_per_page=500&_page=6
http://cataloguekoha.ntrbx.local/api/v1/patrons?_page=7&_per_page=500
http://cataloguekoha.ntrbx.local/api/v1/patrons?_page=8&_per_page=500
http://cataloguekoha.ntrbx.local/api/v1/patrons?_per_page=500&_page=9
http://cataloguekoha.ntrbx.local/api/v1/patrons?_per_page=500&_page=10
http://cataloguekoha.ntrbx.local/api/v1/patrons?_page=11&_per_page=500
http://cataloguekoha.ntrbx.local/api/v1/patrons?_per_page=500&_page=12
http://cataloguekoha.ntrbx.local/api/v1/patrons?_per_page=500&_page=13
http://cataloguekoha.ntrbx.local/api/v1/patrons?_per_page=500&_page=14
http://cataloguekoha.ntrbx.local/api/v1/patrons?_page=15&_per_page=500
http://catalog

In [85]:
df = pd.concat(dfs)
df

,address,address2,altaddress_address,altaddress_address2,altaddress_city,altaddress_country,altaddress_email,altaddress_notes,altaddress_phone,altaddress_postal_code,...,staff_notes,state,statistics_1,statistics_2,street_number,street_type,surname,title,updated_on,userid
0,14 RUE BELL,Appt. 12,,,,,,,,,...,None,595120000011024,None,None,None,None,SEYDI,None,2024-04-03T17:47:07+02:00,X0002404233
1,21 RUE INGRES,chez Mme VANHOVE Laetitia,2024-03-07,message vocal,,,,,,,...,None,5095,None,None,None,None,DUMONT,None,2024-03-23T02:48:03+01:00,X0001659292
2,8 RUE LEVERRIER,Chez Mr HAFIDI Mohamed,,,,,,,,,...,None,18717,None,None,None,None,HAFIDI,None,2024-01-27T17:25:43+01:00,X0002159331
3,20 RUE DUGUESCLIN,Chez Mme. Naïma BOUHAFS,,,,,,,,,...,None,16316,None,None,None,None,BOUHAFS,None,2024-01-27T17:25:43+01:00,X0002438696
4,45 RUE VALLON,Appt 12 Chez Mme Benbahlouli Myriam,2021-01-19,usager joint,2021-02-09,2021-02-09,,C0005382767\r\nC2500004195\r\nC2500012698\r\nC...,,91611,...,None,8446,None,None,None,None,BENBAHLOULI,None,2024-01-27T17:25:43+01:00,X0002315447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,62 rue Henri LORIDANT,,,,,,,,,,...,None,,None,None,None,None,EL BOUAZZATI,None,2025-04-17T14:41:43+02:00,X0002814469
45,2 rue Henri Carette,,,,,,,,,,...,None,,None,None,None,None,GORLIER,None,2025-04-17T15:11:22+02:00,X0002814452
46,1 AlleE Alexandre Desrousseaux,Chez-Mme TERFAIA Lyna,,,,,,,,,...,None,,None,None,None,None,BENAISSA,None,2025-04-17T15:30:27+02:00,X0002814445
47,1 AlleE Alexandre Desrousseaux,Chez-Mme TERFAIA Lyna,,,,,,,,,...,None,,None,None,None,None,BENAISSA,None,2025-04-17T15:32:06+02:00,X0002814438


# Lister les adhérents dont la carte a été créée aujourd'hui

In [116]:
today = get_date_and_time('today')
url = f"{ws_conf['base']}/api/v1/patrons?_per_page=20"
data = json.dumps({"dateenrolled": today })
response = requests.get(url, auth=basic_auth, data=data)
df = pd.DataFrame(response.json())
df['cardnumber']

0     X0002792002
1     X0002813943
2     X0002813950
3     X0002813967
4     X0002813776
5     X0002813783
6     X0002813813
7     X0002813806
8     X0002813790
9     X0002813486
10    X0002813493
11    X0002813530
12    X0002813523
13    X0002813516
14    X0002813639
15    X0002813622
16    X0002814483
17    X0002814476
18    X0002814469
19    X0002814452
Name: cardnumber, dtype: object

# Modifier les données d'un adhérent

In [97]:
patron_id = 63471
url = f"{ws_conf['base']}/api/v1/patrons/{patron_id}"
data = json.dumps({"surname": "PICHENOT",
                   "address": "17 RUE FERDINAND BUISSON",
                   "city": "HAZEBROUCK",
                   "library_id": "MED",
                   "category_id": "MEDP"})
# attention, tous les champs ci-dessus sont obligatoires
response = requests.put(url, auth=basic_auth, data=data)
response

<Response [200]>